<a href="https://colab.research.google.com/github/RaphaelCarvalh/clothing-segmentation-project/blob/main/Executando_o_modelo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Montar o Drive e acessar a pasta compartilhada


In [37]:
from google.colab import drive
import os

# Monta o Drive
drive.mount('/content/drive')

# Pasta fixa para checkpoints
FIXED_DIR = "/content/drive/MyDrive/clothing_segmentation_pasta2/checkpoints"
os.makedirs(FIXED_DIR, exist_ok=True)
print(f"✅ Pasta criada / confirmada: {FIXED_DIR}")

# Caminho para o melhor modelo
BEST_MODEL_PATH = os.path.join(FIXED_DIR, "best_model_final.pt")
print("⚠️ Coloque seu arquivo 'best_model_final.pt' dentro desta pasta:")
print(BEST_MODEL_PATH)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Pasta criada / confirmada: /content/drive/MyDrive/clothing_segmentation_pasta2/checkpoints
⚠️ Coloque seu arquivo 'best_model_final.pt' dentro desta pasta:
/content/drive/MyDrive/clothing_segmentation_pasta2/checkpoints/best_model_final.pt


In [47]:
# Exemplo de uso
FIXED_DIR = "/content/drive/MyDrive/clothing_segmentation_pasta2/provas"
os.makedirs(FIXED_DIR, exist_ok=True)
print(f"✅ Pasta criada / confirmada: {FIXED_DIR}")

✅ Pasta criada / confirmada: /content/drive/MyDrive/clothing_segmentation_pasta2/provas


In [38]:
import os

CHECKPOINT_DIR = "/content/drive/MyDrive/clothing_segmentation_pasta2/checkpoints"

# Cria a pasta caso não exista
os.makedirs(CHECKPOINT_DIR, exist_ok=True)

# Lista o conteúdo
print(f"Conteúdo de {CHECKPOINT_DIR}:")
for item in os.listdir(CHECKPOINT_DIR):
    print("-", item)


Conteúdo de /content/drive/MyDrive/clothing_segmentation_pasta2/checkpoints:
- checkpoint_epoch_1.pt
- checkpoint_epoch_2.pt
- checkpoint_epoch_3.pt
- checkpoint_epoch_4.pt
- checkpoint_epoch_5.pt
- checkpoint_epoch_6.pt
- checkpoint_epoch_7.pt
- checkpoint_epoch_8.pt
- checkpoint_epoch_9.pt
- checkpoint_epoch_10.pt
- checkpoint_epoch_11.pt
- checkpoint_epoch_12.pt
- checkpoint_epoch_13.pt
- checkpoint_epoch_14.pt
- checkpoint_epoch_15.pt
- checkpoint_epoch_16.pt
- checkpoint_epoch_17.pt
- checkpoint_epoch_18.pt
- checkpoint_epoch_19.pt
- checkpoint_epoch_20.pt
- checkpoint_epoch_21.pt
- checkpoint_epoch_22.pt
- checkpoint_epoch_23.pt
- checkpoint_epoch_24.pt
- best_model_final.pt
- log_treinamento.csv


In [44]:
import torch
from torchvision.models.segmentation import deeplabv3_resnet50

# 4️⃣ Definir dispositivo (GPU se disponível)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("🚀 Usando dispositivo:", device)

# 5️⃣ Criar modelo com num_classes correto
num_classes = 5  # ajuste conforme seu treinamento
model = deeplabv3_resnet50(weights=None, aux_loss=True)
model.classifier[-1] = torch.nn.Conv2d(256, num_classes, kernel_size=(1,1))
model.aux_classifier[-1] = torch.nn.Conv2d(256, num_classes, kernel_size=(1,1))
model = model.to(device)

# 6️⃣ Carregar checkpoint
checkpoint = torch.load(BEST_MODEL_PATH, map_location=device)
if "model_state" in checkpoint:
    state_dict = checkpoint["model_state"]
else:
    state_dict = checkpoint

# 7️⃣ Ignorar chaves do aux_classifier caso queira
state_dict = {k: v for k, v in state_dict.items() if not k.startswith("aux_classifier")}
model.load_state_dict(state_dict, strict=False)

model.eval()
print("✅ Modelo carregado e pronto para inferência!")

🚀 Usando dispositivo: cpu
✅ Modelo carregado e pronto para inferência!


In [59]:
from PIL import Image
import torch
from torchvision.models.segmentation import deeplabv3_resnet50
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

device = torch.device("cpu")  # ou "cuda" se disponível

# Carregar modelo
num_classes = 5
model = deeplabv3_resnet50(weights=None)
model.classifier[-1] = torch.nn.Conv2d(256, num_classes, kernel_size=(1,1))
state_dict = torch.load("/content/drive/MyDrive/clothing_segmentation_pasta2/checkpoints/best_model_final.pt", map_location=device)
model.load_state_dict(state_dict, strict=False)  # <- important
model.eval()
model.to(device)

# Transform
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# Carregar imagem
img_path = "/mnt/data/54f7ce4d-aa88-4295-ba69-6fa5fe0bc429.png"
img = Image.open(img_path).convert("RGB")
img_tensor = transform(img).unsqueeze(0).to(device)

# Inferência
with torch.no_grad():
    output = model(img_tensor)['out']
    mask = torch.argmax(output.squeeze(), dim=0).cpu().numpy()

# Redimensionar máscara para tamanho original
mask_img = Image.fromarray(mask.astype(np.uint8)).resize(img.size, resample=Image.NEAREST)
mask_array = np.array(mask_img)

# Visualização
plt.figure(figsize=(10,5))
plt.subplot(1,2,1)
plt.imshow(img)
plt.title("Imagem Original")
plt.subplot(1,2,2)
plt.imshow(mask_array, cmap="tab20")
plt.title("Máscara")
plt.show()


FileNotFoundError: [Errno 2] No such file or directory: '/mnt/data/54f7ce4d-aa88-4295-ba69-6fa5fe0bc429.png'